In [1]:
import numpy as np
import tensorflow as tf
print(tf.__version__)
%load_ext line_profiler

2.3.0


In [2]:
USE_PTB = True
USE_NATIVE=True

# Text to process

### Test sample text

In [3]:
text = "The fool doth think he is wise, but the wise man knows himself to be a fool."
text = "To be, or not to be, that is the question"

## PTB (Penn Treebank) 

In [4]:
#coding: utf-8
import sys
import os
sys.path.append('..')
try:
    import urllib.request
except ImportError:
    raise ImportError('Use Python3!')
import pickle
import numpy as np


url_base = 'https://raw.githubusercontent.com/tomsercu/lstm/master/data/'
key_file = {
    'train':'ptb.train.txt',
    'test':'ptb.test.txt',
    'valid':'ptb.valid.txt'
}
save_file = {
    'train':'ptb.train.npy',
    'test':'ptb.test.npy',
    'valid':'ptb.valid.npy'
}
vocab_file = 'ptb.vocab.pkl'

#dataset_dir = os.path.dirname(os.path.abspath(__file__))
dataset_dir = os.path.dirname(os.path.abspath("/home/oonisim/dataset/hoge"))
print(dataset_dir)

def _download(file_name):
    file_path = dataset_dir + '/' + file_name
    if os.path.exists(file_path):
        return

    print('Downloading ' + file_name + ' ... ')

    try:
        urllib.request.urlretrieve(url_base + file_name, file_path)
    except urllib.error.URLError:
        import ssl
        ssl._create_default_https_context = ssl._create_unverified_context
        urllib.request.urlretrieve(url_base + file_name, file_path)

    print('Done')


def load_text():
    data_type = 'train'
    file_name = key_file[data_type]
    file_path = dataset_dir + '/' + file_name
    vocab_path = dataset_dir + '/' + vocab_file

    if not os.path.exists(file_path):
        _download(file_name)

    text = open(file_path).read().replace('\n', '<eos>').strip()
    return(text)
    
def load_vocab():
    vocab_path = dataset_dir + '/' + vocab_file

    if os.path.exists(vocab_path):
        with open(vocab_path, 'rb') as f:
            word_to_id, id_to_word = pickle.load(f)
        return word_to_id, id_to_word

    word_to_id = {}
    id_to_word = {}
    data_type = 'train'
    file_name = key_file[data_type]
    file_path = dataset_dir + '/' + file_name

    _download(file_name)

    words = open(file_path).read().replace('\n', '<eos>').strip().split()

    for i, word in enumerate(words):
        if word not in word_to_id:
            tmp_id = len(word_to_id)
            word_to_id[word] = tmp_id
            id_to_word[tmp_id] = word

    with open(vocab_path, 'wb') as f:
        pickle.dump((word_to_id, id_to_word), f)

    return word_to_id, id_to_word


def load_data(data_type='train'):
    '''
        :param data_type: データの種類：'train' or 'test' or 'valid (val)'
        :return:
    '''
    if data_type == 'val': data_type = 'valid'
    save_path = dataset_dir + '/' + save_file[data_type]

    word_to_id, id_to_word = load_vocab()

    if os.path.exists(save_path):
        corpus = np.load(save_path)
        return corpus, word_to_id, id_to_word

    file_name = key_file[data_type]
    file_path = dataset_dir + '/' + file_name
    _download(file_name)

    words = open(file_path).read().replace('\n', '<eos>').strip().split()
    corpus = np.array([word_to_id[w] for w in words])

    np.save(save_path, corpus)
    return corpus, word_to_id, id_to_word



/home/oonisim/dataset


# Word indexing
Assign a numerical id to each word.

## Proprietary indexing

In [5]:
def preprocess(text):
    """
    Args: 
        text: A string including sentences to process.
    Returns:
        corpus: 
            A numpy array of word indices to every word in the originlal text as as they appear in the text.
            The objective of corpus is to preserve the original text but as numerical indices.
        word_to_id: A dictionary to map a word to a word index
        id_to_word: A dictionary to map a word index to a word
    """
    text = text.lower()
    text = text.replace('.', ' .')
    words = text.split(' ')

    word_to_id = {}
    id_to_word = {}
    for word in words:
        if word not in word_to_id:
            new_id = len(word_to_id)
            word_to_id[word] = new_id
            id_to_word[new_id] = word

    corpus = np.array([word_to_id[w] for w in words])

    return corpus, word_to_id, id_to_word

In [6]:
if USE_NATIVE:
    if USE_PTB:
        (corpus, word_to_id, id_to_word)= load_data('train')
    else:
        (corpus, word_to_id, id_to_word) = preprocess(text)

    print(len(corpus))
    print(len(word_to_id))
    vocab_size = max(word_to_id.values()) + 1
    print(vocab_size)
    
if not USE_PTB:
    print(word_to_id)

929589
10000
10000


## Tensorflow Tokenizer indexing

In [12]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(oov_token="<OOV>")

USE_TENSORFLOW = (not USE_NATIVE)
if USE_TENSORFLOW:
    if USE_PTB:
        text = load_text()
    else: 
        text = text

    # Each text in "texts" is a complete document as one string, 
    # e.g "To be or not to be, that is the question."
    texts = [ text ]   

    # fit_on_texts() processes multiple documents and handles all words in all the documents.
    tokenizer.fit_on_texts(texts)
    word_to_id = tokenizer.word_index

    # texts_to_sequences() ruturns word index sequence of each document in "texts".
    sequences = (tokenizer.texts_to_sequences(texts))

    # corpus is word index sequence for a single document loaded by load_text().
    corpus = sequences[0]

    print(len(sequences))
    print(len(corpus))
    print(len(word_to_id))
    
    # Index of tokenizer.word_index starts at 1, NOT 0.
    # e.g. {'<OOV>': 1, 'the': 2, 'fool': 3, 'wise': 4, 'doth': 5, ...}
    vocab_size = max(word_to_id.values()) + 1
    print(vocab_size)

# Co-occurrence vector

In [13]:
def create_co_matrix(corpus, vocab_size, window_size=1):
    '''Generate co-occurreance matrix for the corpus.

    :param corpus:
    :param vocab_size:The number of unique words in the corpus. 
    :param window_size: 
        The number of words either left or right of the word to count co-occurreances, which is (context_ize / 2)
    :return: co-occurrence matrix
    '''
    corpus_size = len(corpus)
    co_matrix = np.zeros((vocab_size, vocab_size), dtype=np.int32)

    for idx, word_id in enumerate(corpus):
        for i in range(1, window_size + 1):
            left_idx = idx - i
            right_idx = idx + i

            if left_idx >= 0:
                left_word_id = corpus[left_idx]
                co_matrix[word_id, left_word_id] += 1

            if right_idx < corpus_size:
                right_word_id = corpus[right_idx]
                co_matrix[word_id, right_word_id] += 1

    return co_matrix

In [14]:
#%%timeit -n 1 
com1 = create_co_matrix(corpus, vocab_size, 1)
print(com1)
#print(corpus)

[[0 2 0 1 0 0 0 0]
 [2 0 1 0 1 0 0 0]
 [0 1 0 1 0 0 0 0]
 [1 0 1 0 0 0 0 0]
 [0 1 0 0 0 1 0 0]
 [0 0 0 0 1 0 1 0]
 [0 0 0 0 0 1 0 1]
 [0 0 0 0 0 0 1 0]]


In [ ]:
%lprun -T create_co_matrix.log -f create_co_matrix create_co_matrix(corpus, vocab_size, 1)
print(open('create_co_matrix.log', 'r').read())

## numpy vectorized way

In [7]:
def debug(corpus, index, stride, flag=False):
    if not flag:
        return
    
    n = len(corpus)
    print("word is {} and context is {}".format(
        id_to_word[corpus[index]],
        [ id_to_word[i] for i in corpus[max(0, (index-stride) +1) : min((index+stride) +1, n)]]
    ))

In [8]:
def create_cooccurrence_matrix(sequence, vocabrary_size, context_size=3):
    """
    Args: 
        sequence: word index sequence of the original corpus text
        vocabrary_size: number of words in the vocabrary (same with co-occurrence vector size)
        context_size: context (N-gram size N) within to check co-occurrences.
    Returns:
        co_occurrence matrix
    """
    n = sequence_size = len(sequence)
    co_occurrence_matrix = np.zeros((vocabrary_size, vocabrary_size), dtype=np.int32)

    stride = int((context_size - 1)/2 )
    assert(n > stride), "sequence_size {} is less than/equal to stride {}".format(
        n, stride
    )

    """
    # Handle position=slice(0 : (stride-1) +1),       co-occurrences=slice(max(0, position-stride): min((position+stride),n-1) +1)
    # Handle position=slice((n-1-stride) : (n-1) +1), co-occurrences=slice(max(0, position-stride): min((position+stride),n-1) +1)
    indices = [*range(0, (stride-1) +1), *range((n-1)-stride +1, (n-1) +1)]
    #print(indices)
    
    for position in indices:
        debug(sequence, position, stride, False)
        co_occurrence_matrix[
            sequence[position],                                             # position to the word
            sequence[max(0, position-stride) : min((position+stride),n-1) +1]  # indices to co-occurence words 
        ] += 1

    
    # Handle position=slice(stride, ((sequence_size-1) - stride) +1)
    for position in range(stride, (sequence_size-1) - stride + 1):        
        co_occurrence_matrix[
            sequence[position],                                 # position to the word
            sequence[(position-stride) : (position + stride + 1)]  # indices to co-occurence words 
        ] += 1
    """        
    
    for position in range(0, n):        
        co_occurrence_matrix[
            sequence[position],                                                # position  to the word
            sequence[max(0, position-stride) : min((position+stride),n-1) +1]  # positions to co-occurence words 
        ] += 1

    np.fill_diagonal(co_occurrence_matrix, 0)
    
    return co_occurrence_matrix

In [19]:
#%%timeit -n 1 
com2 = create_cooccurrence_matrix(corpus, vocab_size, 3)
#print((com2))
assert (com1==com2).all()
print(com2)

[[0 2 0 1 0 0 0 0]
 [2 0 1 0 1 0 0 0]
 [0 1 0 1 0 0 0 0]
 [1 0 1 0 0 0 0 0]
 [0 1 0 0 0 1 0 0]
 [0 0 0 0 1 0 1 0]
 [0 0 0 0 0 1 0 1]
 [0 0 0 0 0 0 1 0]]


In [9]:
%lprun -T create_cooccurrence_matrix.log -f create_cooccurrence_matrix create_cooccurrence_matrix(corpus, vocab_size, 3)
print(open('create_cooccurrence_matrix.log', 'r').read())


*** Profile printout saved to text file 'create_cooccurrence_matrix.log'. 
Timer unit: 1e-06 s

Total time: 23.0015 s
File: <ipython-input-8-27f5e530d4ff>
Function: create_cooccurrence_matrix at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def create_cooccurrence_matrix(sequence, vocabrary_size, context_size=3):
     2                                               """
     3                                               Args: 
     4                                                   sequence: word index sequence of the original corpus text
     5                                                   vocabrary_size: number of words in the vocabrary (same with co-occurrence vector size)
     6                                                   context_size: context (N-gram size N) within to check co-occurrences.
     7                                               Returns:
     8                                       

In [ ]:
def create_cooccurrence_matrix_2(corpus, vector_size, context_size=3):
    """
    Create a co-occurrence matrix for the copus with two paths.
        1. Handle co-occurrences to the right of the word in the context.
        2. Handle co-occurrences to the left  of the word in the context.
    Args: 
        copus: Original text as word indices
        vector_size: Co-occurrence vector size (including the word itself)
        context_size: Context (N-gram) size N
        
    """
    corpus_size = len(corpus)
    co_ccurrence_matrix = np.zeros((vector_size, vector_size), dtype=np.int32)

    stride = int((context_size - 1)/2 )

    # Handle right stride
    for index in range(0, (corpus_size-1) - stride +1):
        debug(corpus, index, stride)
        co_ccurrence_matrix[
            corpus[index],                       # index to the word
            corpus[index+1 : (index+stride) +1]  # indices to right co-occurance words excluding word itself
        ] += 1
        
    # Handle left stride
    for index in range(stride, (corpus_size-1) +1):
        debug(corpus, index, stride) 
        co_ccurrence_matrix[
            corpus[index],                      # index to the word
            corpus[(index-stride) : index]      # indices to left co-occurance words excluding word itself
        ] += 1

    return co_ccurrence_matrix

In [ ]:
#%%timeit -n 1 
com3 = create_cooccurrence_matrix_2(corpus, vocab_size, 3)
#print(com3)
assert (com1==com3).all()

In [ ]:
%lprun -T create_cooccurrence_matrix_2.log -f create_cooccurrence_matrix_2 create_cooccurrence_matrix_2(corpus, vocab_size, 3)
print(open('create_cooccurrence_matrix_2.log', 'r').read())

# Combined 

In [85]:
import numpy as np
 
def preprocess(text):
    """
    Args:
        text: A string including sentences to process.
    Returns:
        corpus:
            A numpy array of word indices to every word in the originlal text as as they appear in the text.
            The objective of corpus is to preserve the original text but as numerical indices.
        word_to_id: A dictionary to map a word to a word index
        id_to_word: A dictionary to map a word index to a word
    """
    text = text.lower()
    text = text.replace('.', ' .')
    words = text.split(' ')
 
    word_to_id = {}
    id_to_word = {}
    for word in words:
        if word not in word_to_id:
            new_id = len(word_to_id)
            word_to_id[word] = new_id
            id_to_word[new_id] = word
 
    corpus = np.array([word_to_id[w] for w in words])
 
    return corpus, word_to_id, id_to_word
 

def create_cooccurrence_matrix(sequence, vocabrary_size, context_size=3):
    """
    Args:
        sequence: word index sequence of the original corpus text
        vocabrary_size: number of words in the vocabrary (same with co-occurrence vector size)
        context_size: context (N-gram size N) within to check co-occurrences.

    """
    n = sequence_size = len(sequence)
    co_ccurrence_matrix = np.zeros((vocabrary_size, vocabrary_size), dtype=np.int32)

    stride = int((context_size - 1)/2 )
    assert(n > stride), "sequence_size {} is less than/equal to stride {}".format(
        n, stride
    ) 

    # For each position, the co_occurrence_indices is (1, stride), ..., (1, 2 * stride)
    position = 0
    indices = np.array([
        [
            sequence[position],                                                # position  to the word
            sequence[max(0, position-stride) : min((position+stride),n-1) +1]  # positions to co-occurance words
        ]]
    )
    for position in range(0, n):
        co_occurrence_indices = np.array([
            [
                sequence[position],                                                # position  to the word
                sequence[max(0, position-stride) : min((position+stride),n-1) +1]  # positions to co-occurance words
            ]]
        )
        print("indices \n{}".format(indices))
        print("indices shape {}\n".format(indices.shape))
        print("co_occurrence_indices \n{}".format(co_occurrence_indices))
        print("co_occurrence_indices shape {}\n".format(co_occurrence_indices.shape))
        indices = np.append(
            indices,
            co_occurrence_indices,
            axis=0
        )

        
    print("-------------------------------")
    print(indices[0])
    indices = np.delete(indices, 0)
    print("-------------------------------")
    print(indices)
    print(indices.shape)
    print("Updating the co_occurrence_matrix: indices \n{} \nindices.dtype {}".format(
        indices,
        indices.dtype
    ))
    co_ccurrence_matrix[
        indices
    ] += 1
 
    np.fill_diagonal(co_ccurrence_matrix, 0)
     
    return co_ccurrence_matrix



corpus = "To be, or not to be, that is the question"
 
sequence, word_to_id, id_to_word = preprocess(corpus)
vocabrary_size = max(word_to_id.values()) + 1
create_cooccurrence_matrix(sequence, vocabrary_size , 3)

In [88]:
import numpy as np
 
def preprocess(text):
    """
    Args:
        text: A string including sentences to process.
    Returns:
        corpus:
            A numpy array of word indices to every word in the originlal text as as they appear in the text.
            The objective of corpus is to preserve the original text but as numerical indices.
        word_to_id: A dictionary to map a word to a word index
        id_to_word: A dictionary to map a word index to a word
    """
    text = text.lower()
    text = text.replace('.', ' .')
    words = text.split(' ')
 
    word_to_id = {}
    id_to_word = {}
    for word in words:
        if word not in word_to_id:
            new_id = len(word_to_id)
            word_to_id[word] = new_id
            id_to_word[new_id] = word
 
    corpus = np.array([word_to_id[w] for w in words])
 
    return corpus, word_to_id, id_to_word
 

def create_cooccurrence_matrix(sequence, vocabrary_size, context_size=3):
    """
    Args:
        sequence: word index sequence of the original corpus text
        vocabrary_size: number of words in the vocabrary (same with co-occurrence vector size)
        context_size: context (N-gram size N) within to check co-occurrences.

    """
    n = sequence_size = len(sequence)
    co_ccurrence_matrix = np.zeros((vocabrary_size, vocabrary_size), dtype=np.int32)

    stride = int((context_size - 1)/2 )
    assert(n > stride), "sequence_size {} is less than/equal to stride {}".format(
        n, stride
    ) 

    position = 0
    indices = np.array([
        [
            sequence[0],                                         # position  to the word
            sequence[max(0, 0-stride) : min((0+stride),n-1) +1]  # positions to co-occurrencewords
        ]]
    )
    assert n > 1
    for position in range(1, n):
        co_occurrence_indices = np.array([
            [
                sequence[position],                                                # position  to the word
                sequence[max(0, position-stride) : min((position+stride),n-1) +1]  # positions to co-occurrencewords
            ]]
        )
        indices = np.append(
            indices,
            co_occurrence_indices,
            axis=0
        )

    print("Updating the co_occurrence_matrix: indices \n{} \nindices.dtype {}".format(
        indices,
        indices.dtype
    ))
    co_ccurrence_matrix[
        indices
    ] += 1
 
    np.fill_diagonal(co_ccurrence_matrix, 0)
     
    return co_ccurrence_matrix



corpus = "To be, or not to be, that is the question"
 
sequence, word_to_id, id_to_word = preprocess(corpus)
vocabrary_size = max(word_to_id.values()) + 1
create_cooccurrence_matrix(sequence, vocabrary_size , 3)

Updating the co_occurrence_matrix: indices 
[[0 array([0, 1])]
 [1 array([0, 1, 2])]
 [2 array([1, 2, 3])]
 [3 array([2, 3, 0])]
 [0 array([3, 0, 1])]
 [1 array([0, 1, 4])]
 [4 array([1, 4, 5])]
 [5 array([4, 5, 6])]
 [6 array([5, 6, 7])]
 [7 array([6, 7])]] 
indices.dtype object


<ipython-input-88-d9b081bf2f1a>:48: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  indices = np.array([
<ipython-input-88-d9b081bf2f1a>:56: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  co_occurrence_indices = np.array([


IndexError: arrays used as indices must be of integer (or boolean) type